## CS 6120: Natural Language Processing - Prof. Ahmad Uzair

### Assignment 4: Parts of Speech Tagging
### Total Points: 100 points

In Assignment 4, you will implement Viterbi Algorithm based on Hidden Markov Model and RNNs with PyTorch , for Part-of-speech tagging (the process of assigning a part-of-speech tag (Noun, Verb, Adjective...) to each word in an input text). We recommend you to start this assignment a little early and fully understand these algorithms before jumping into coding. 

## Parts of speech tagging. 

Parts of Speech Tagging is the process of the assigning a parts of speech tag (noun, adjective etc..,) to each word in the input sentence. <br>

In this question we will be implementing a Hidden Markov Model (HMM) with a Viterbi decoder
- The HMM is one of the most commonly used algorithms in Natural Language Processing, and is a foundation to many deep learning techniques you will see in this specialization. 
- In addition to parts-of-speech tagging, HMM is used in speech recognition, speech synthesis, etc. 
- By completing this part of the assignment you will get a 95% accuracy on the same dataset you used in Part 1.

The Markov Model contains a number of states and the probability of transition between those states. 
- In this case, the states are the parts-of-speech. 
- A Markov Model utilizes a transition matrix, `A`. 
- A Hidden Markov Model adds an observation or emission matrix `B` which describes the probability of a visible observation when we are in a particular state. 
- In this case, the emissions are the words in the corpus
- The state, which is hidden, is the POS tag of that word.

### About the Dataset: <br>

For this task we will be using tagged datasets collected from Wall Street Journal. <br>

The file train.pos will be used for training and test.pos will for testing. Along with these two we will be providing vocab.txt the words in this file are the words from the training set that were used two or more times.<br>

The dataset will contain different tags like JJ which means adjective, DT means determiner etc.., for better understaning of the tags refer to [this link](http://relearn.be/2015/training-common-sense/sources/software/pattern-2.6-critical-fork/docs/html/mbsp-tags.html)

In [2]:
##############################################################################################
##### Dont change anything in this code cell , only change the data paths accordingly  #######
##############################################################################################

#Importing necessary packages

import pandas as pd
from collections import defaultdict
import math
import numpy as np
import string

# Punctuation characters
punct = set(string.punctuation)

# Morphology rules used to assign unknown word tokens
noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]

# Utility functions which we further need

def get_word_tag(line, vocab): 
    if not line.split():
        word = "--n--"
        tag = "--s--"
        return word, tag
    else:
        word, tag = line.split()
        if word not in vocab: 
            # Handle unknown words
            word = assign_unk(word)
        return word, tag
    return None 


def preprocess(vocab, data_fp):
    """
    Preprocess data
    """
    orig = []
    prep = []

    # Read data
    with open(data_fp, "r") as data_file:

        for cnt, word in enumerate(data_file):

            # End of sentence
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Handle unknown words
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep


def assign_unk(tok):
    """
    Assign unknown word tokens
    """
    # Digits
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Punctuation
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Upper-case
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Nouns
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Verbs
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Adjectives
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Adverbs
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"

    return "--unk--"


# for viterbi
def compute_accuracy(pred, y):
    '''
    Input: 
        pred: a list of the predicted parts-of-speech 
        y: a list of lines where each word is separated by a '\t' (i.e. word \t tag)
    Output: 
        
    '''
    num_correct = 0
    total = 0
    
    # Zip together the prediction and the labels
    for prediction, y in zip(pred, y):
        ### START CODE HERE (Replace instances of 'None' with your code) ###
        # Split the label into the word and the POS tag
        word_tag_tuple = y.split()
        
        # Check that there is actually a word and a tag
        # no more and no less than 2 items
        if len(word_tag_tuple)!=2: # complete this line
            continue 

        # store the word and tag separately
        word, tag = word_tag_tuple
        
        # Check if the POS tag label matches the prediction
        if prediction == tag: # complete this line
            
            # count the number of times that the prediction
            # and label match
            num_correct += 1
            
        # keep track of the total number of examples (that have valid labels)
        total += 1
        
        ### END CODE HERE ###
    return num_correct/total


# load in the training corpus
with open("training.pos.txt", 'r') as f:
    training_corpus = f.readlines()

# read the vocabulary data, split by each line of text, and save the list
with open("vocab.txt", 'r') as f:
    voc_l = f.read().split('\n')

# vocab: dictionary that has the index of the corresponding words
vocab = {} 

# Get the index of the corresponding words. 
for i, word in enumerate(sorted(voc_l)): 
    vocab[word] = i       
    

cnt = 0
for k,v in vocab.items():
    cnt += 1
    if cnt > 20:
        break

# load in the test corpus
with open("test.pos.txt", 'r') as f:
    y = f.readlines()

#corpus without tags, preprocessed
_, prep = preprocess(vocab, "test.words.txt")



### Task 1.1 : Transisiont, Emission metrices (10 Points)

In this task we are expected to build a function which takes training_corpus as input and return transition counts, emission counts and tag counts. <br> 



1. `Tranition count`: maps prev_tag, tag) to the number of times it has appeared.
2. `Emission_counts`: maps (tag, word) to the number of times it appeared.
3. `Tag_counts`: maps (tag) to the number of times it has occured.



In [ ]:
def create_dictionaries(training_corpus, vocab):
    """
    Input: 
        training_corpus: a corpus where each line has a word followed by its tag.
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output: 
        emission_counts: a dictionary where the keys are (tag, word) and the values are the counts
        transition_counts: a dictionary where the keys are (prev_tag, tag) and the values are the counts
        tag_counts: a dictionary where the keys are the tags and the values are the counts
    """
    
    # initialize the dictionaries using defaultdict
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    # Initialize "prev_tag" (previous tag) with the start state, denoted by '--s--'
    prev_tag = '--s--' 
    
    # use 'i' to track the line number in the corpus
    i = 0 
    
    # Each item in the training corpus contains a word and its POS tag
    # Go through each word and its tag in the training corpus
    for word_tag in training_corpus:
        
        # Increment the word_tag count
        i += 1
            
        ### START CODE HERE (Replace instances of 'None' with your code) ###
        # get the word and tag using the get_word_tag helper function
        word, tag = get_word_tag(word_tag, vocab)
        
        # Increment the transition count for the previous word and tag
        transition_counts[(prev_tag, tag)] += 1
            
        # Increment the emission count for the tag and word
        emission_counts[(tag, word)] += 1
            
        # Increment the tag count
        tag_counts[tag] += 1
            
        # Set the previous tag to this tag (for the next iteration of the loop)
        prev_tag = tag
        ### END CODE HERE ###
        
    return emission_counts, transition_counts, tag_counts

In [ ]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

In [ ]:
# get all the POS states. States are parts of speech designation found in the training dataset.
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)

Number of POS tags (number of 'states'): 46
View these POS tags (states)
['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


##### Expected Output

```CPP
Number of POS tags (number of 'states'46
View these states
['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
```

In [ ]:
print("transition examples: ")
for ex in list(transition_counts.items())[:3]:
    print(ex)
print()

print("emission examples: ")
for ex in list(emission_counts.items())[:3]:
    print (ex)
print()

print("ambiguous word example: ")
for tup,cnt in list(emission_counts.items())[:3]:
    print(tup, cnt) 

transition examples: 
(('--s--', 'IN'), 5050)
(('IN', 'DT'), 32364)
(('DT', 'NNP'), 9044)

emission examples: 
(('IN', 'In'), 1735)
(('DT', 'an'), 3142)
(('NNP', 'Oct.'), 317)

ambiguous word example: 
('IN', 'In') 1735
('DT', 'an') 3142
('NNP', 'Oct.') 317



##### Expected Output

```CPP
transition examples: 
(('--s--', 'IN'), 5050)
(('IN', 'DT'), 32364)
(('DT', 'NNP'), 9044)

emission examples: 
(('IN', 'In'), 1735)
(('DT', 'an'), 3142)
(('NNP', 'Oct.'), 317)

ambiguous word example: 
('IN', 'In') 1735
('DT', 'an') 3142
('NNP', 'Oct.') 317
```

### Task 1.2: Predict (10 Points)

You need to complete the `predict_pos` function below which takes preprocessed test corpus (prep), Original tagged test corpus `y`, emission counts, vocab and states. <br>

Ultimately in this function for a given preprocessed test corpus, you will assign a parts-of-speech tag to every word in that corpus. Using the original tagged test corpus, you will then compute what percent of the tags you got correct. <br>

In [ ]:
def predict_pos(prep, y, emission_counts, vocab, states):
    '''
    Input: 
        prep: a preprocessed version of 'y'. A list with the 'word' component of the tuples.
        y: a corpus composed of a list of tuples where each tuple consists of (word, POS)
        emission_counts: a dictionary where the keys are (tag,word) tuples and the value is the count
        vocab: a dictionary where keys are words in vocabulary and value is an index
        states: a sorted list of all possible tags for this assignment
    Output: 
        accuracy: Number of times you classified a word correctly
    '''
    
    # Initialize the number of correct predictions to zero
    num = 0
    # Get the (tag, word) tuples, stored as a set
    all_words = set(emission_counts.keys())
    
    # Get the number of (word, POS) tuples in 'y'
    den = len(y)

    
    for word, tuplY in zip(prep, y): 

        # Split the (word, POS) string into a list.
        tuplY_list = tuplY.split()
        true_label = tuplY_list[1]

        posF = ''

        countF = 0
      
        if word in vocab:
            for posT in states:
                        
                # define the key as the tuple containing the POS and word
                key = (posT, word)

                # check if the (pos, word) key exists in the emission_counts dictionary
                if key in emission_counts:

                # get the emission count of the (pos,word) tuple 
                    count = emission_counts[key]

                    # keep track of the POS with the largest count
                    if count > countF: 

                        # update the final count and POS
                        countF = count
                        posF = posT

            
            if posF == true_label:
                # Updating the number of correct predictions
                num += 1
   
       
    ### END CODE HERE ###
    accuracy = num / den
    return accuracy

In [ ]:
accuracy_predict_pos = predict_pos(prep, y, emission_counts, vocab, states)
print(f"Accuracy of prediction using predict_pos is {accuracy_predict_pos:.4f}")

Accuracy of prediction using predict_pos is 0.8889


##### Expected Output

```CPP
Accuracy of prediction using predict_pos is 0.8889
```

### Task 1.3 Building Hidden Markov Models for POS. (10 Points)

**Hidden Markov Models** (HMMs) are a class of probabilistic graphical model that allow us to predict a sequence of unknown (hidden) variables from a set of observed variables. <br>

The Markov Model contains a number of states and the probability of transition between those states. 
- In this case, the states are the parts-of-speech. 
- A Markov Model utilizes a transition matrix, `A`. 
- A Hidden Markov Model adds an observation or emission matrix `B` which describes the probability of a visible observation when we are in a particular state. 
- In this case, the emissions are the words in the corpus
- The state, which is hidden, is the POS tag of that word.


### Creating the 'A' transition probabilities matrix

We will be using Smoothing to compute the matrix. 

The smoothing was done as follows: 

$$ P(t_i | t_{i-1}) = \frac{C(t_{i-1}, t_{i}) + \alpha }{C(t_{i-1}) +\alpha * N}$$

- $N$ is the total number of tags
- $C(t_{i-1}, t_{i})$ is the count of the tuple (previous POS, current POS) in `transition_counts` dictionary.
- $C(t_{i-1})$ is the count of the previous POS in the `tag_counts` dictionary.
- $\alpha$ is a smoothing parameter.

In [ ]:
# GRADED FUNCTION: create_transition_matrix
def create_transition_matrix(alpha, tag_counts, transition_counts):
    ''' 
    Input: 
        alpha: number used for smoothing
        tag_counts: a dictionary mapping each tag to its respective count
        transition_counts: transition count for the previous word and tag
    Output:
        A: matrix of dimension (num_tags,num_tags)
    '''
    
    num_tags = len(tag_counts)

    A = np.zeros((num_tags, num_tags))
    # Create sorted version of the tag's list
    sorted_tags = sorted(states)
    

    ### Your Code Goes Here ###
    
    
    for i in range(num_tags):
        for j in range(num_tags):
            count = 0
            # retreiving the tags at position i,j 
            key = (sorted_tags[i],sorted_tags[j])

            # Looking for (prev POS, current POS) in the transition counts
            if key in transition_counts: 
                
                #getting the count for the key
                count = transition_counts[key]
                
            # previous tag count
            count_prev_tag = tag_counts[key[0]]
            
            A[i,j] = (count + alpha)/(count_prev_tag + num_tags*alpha)
            
    return A

In [ ]:
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)
# Testing your function
print(f"A at row 0, col 0: {A[0,5]:.9f}")
print(f"A at row 3, col 1: {A[3,6]:.4f}")

print("View a subset of transition matrix A")
A_sub = pd.DataFrame(A[10:15,10:15], index=states[10:15], columns = states[10:15] )
print(A_sub)

A at row 0, col 0: 0.007047013
A at row 3, col 1: 0.0000
View a subset of transition matrix A
          CD        DT            EX        FW        IN
CD  0.201542  0.028850  2.734628e-08  0.000055  0.089997
DT  0.022922  0.001576  1.221866e-08  0.000257  0.009665
EX  0.000001  0.002319  1.158687e-06  0.000001  0.000001
FW  0.000004  0.008550  4.272664e-06  0.239273  0.029913
IN  0.059328  0.328388  1.582898e-03  0.000203  0.020415


#### Expected Output: 

```CPP
A at row 0, col 0: 0.007047013
A at row 3, col 1: 0.0000
View a subset of transition matrix A
          CD        DT            EX        FW        IN
CD  0.201542  0.028850  2.734628e-08  0.000055  0.089997
DT  0.022922  0.001576  1.221866e-08  0.000257  0.009665
EX  0.000001  0.002319  1.158687e-06  0.000001  0.000001
FW  0.000004  0.008550  4.272664e-06  0.239273  0.029913
IN  0.059328  0.328388  1.582898e-03  0.000203  0.020415
```

#### Creating 'B' emission probabilities matrix

Now you will create the `B` transition matrix which computes the emission probability. 

You will use smoothing as defined below: 

$$P(w_i | t_i) = \frac{C(t_i, word_i)+ \alpha}{C(t_{i}) +\alpha * N}\$$

- $C(t_i, word_i)$ is the number of times $word_i$ was associated with $tag_i$ in the training data (stored in `emission_counts` dictionary).
- $C(t_i)$ is the number of times $tag_i$ was in the training data (stored in `tag_counts` dictionary).
- $N$ is the number of words in the vocabulary
- $\alpha$ is a smoothing parameter. 

The matrix `B` is of dimension (num_tags, N), where num_tags is the number of possible parts-of-speech tags. 

In [ ]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: create_emission_matrix

def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    '''
    Input: 
        alpha: tuning parameter used in smoothing 
        tag_counts: a dictionary mapping each tag to its respective count
        emission_counts: a dictionary where the keys are (tag, word) and the values are the counts
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output:
        B: a matrix of dimension (num_tags, len(vocab))
    '''
    num_tags = len(tag_counts)
    B = np.zeros((num_tags, len(vocab)))

    # Create sorted version of the tag's list
    sorted_tags = sorted(tag_counts.keys())
    
    # Go through each row (POS tags)
    for i in range(num_tags):
        for j in range(len(vocab)):
            # Initializing the emission count to zero
            count = 0      
            # Retreiving the (POS tag, word) for the respective row and column
            key =  (sorted_tags[i],vocab[j])

            # looking for the (POS tag, word) tuple in emission counts
            if key in emission_counts:
                count = emission_counts[key]
                
            # Get the count of the POS tag
            count_POS_tag = tag_counts[key[0]]

            B[i,j] = (count + alpha)/(count_POS_tag + alpha * len(vocab))
    
    return B

In [ ]:
# creating your emission probability matrix. this takes a few minutes to run. 
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))

print(f"View Matrix position at row 0, column 0: {B[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {B[3,1]:.9f}")

# Try viewing emissions for a few words in a sample dataframe
cidx  = ['725','adroitly','engineers', 'promoted', 'synergy']

# Get the integer ID for each word
cols = [vocab[a] for a in cidx]

# Choose POS tags to show in a sample dataframe
rvals =['CD','NN','NNS', 'VB','RB','RP']

# For each POS tag, get the row number from the 'states' list
rows = [states.index(a) for a in rvals]

# Get the emissions for the sample of words, and the sample of POS tags
B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns = cidx )
print(B_sub)

View Matrix position at row 0, column 0: 0.000006032
View Matrix position at row 3, column 1: 0.000000720
              725      adroitly     engineers      promoted       synergy
CD   8.201296e-05  2.732854e-08  2.732854e-08  2.732854e-08  2.732854e-08
NN   7.521128e-09  7.521128e-09  7.521128e-09  7.521128e-09  2.257091e-05
NNS  1.670013e-08  1.670013e-08  4.676203e-04  1.670013e-08  1.670013e-08
VB   3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08
RB   3.226454e-08  6.456135e-05  3.226454e-08  3.226454e-08  3.226454e-08
RP   3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07


#### Expected Output: 

```CPP
View Matrix position at row 0, column 0: 0.000006032
View Matrix position at row 3, column 1: 0.000000720
              725      adroitly     engineers      promoted       synergy
CD   8.201296e-05  2.732854e-08  2.732854e-08  2.732854e-08  2.732854e-08
NN   7.521128e-09  7.521128e-09  7.521128e-09  7.521128e-09  2.257091e-05
NNS  1.670013e-08  1.670013e-08  4.676203e-04  1.670013e-08  1.670013e-08
VB   3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08
RB   3.226454e-08  6.456135e-05  3.226454e-08  3.226454e-08  3.226454e-08
RP   3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07
```

### Task 1.4 Viterbi Algorithm

In this part of the assignment you will implement the Viterbi algorithm. Specifically, you will use your two matrices, `A` and `B` to compute the Viterbi algorithm. 

We have decomposed this process into three main parts. 

* **Initialization** - In this part you initialize the `best_paths` and `best_probabilities` matrices that you will be populating in `feed_forward`.
* **Feed forward** - At each step, you calculate the probability of each path happening and the best paths up to that point. 
* **Feed backward**: This allows you to find the best path with the highest probabilities. 

### Task 1.4.1 Initialization 
Write a program below that initializes the `best_probs` and the `best_paths` matrix. 

Both matrices will be initialized to zero except for column zero of `best_probs`.  
- Column zero of `best_probs` is initialized with the assumption that the first word of the corpus was preceded by a start token ("--s--"). 
- This allows you to reference the **A** matrix for the transition probability

Here is how to initialize column 0 of `best_probs`:
- The probability of the best path going from the start index to a given POS tag indexed by integer $i$ is denoted by $\textrm{best_probs}[s_{idx}, i]$.
- This is estimated as the probability that the start tag transitions to the POS denoted by index $i$: $\mathbf{A}[s_{idx}, i]$ AND that the POS tag denoted by $i$ emits the first word of the given corpus, which is $\mathbf{B}[i, vocab[corpus[0]]]$.
- Note that vocab[corpus[0]] refers to the first word of the corpus (the word at position 0 of the corpus). 
- **vocab** is a dictionary that returns the unique integer that refers to that particular word.


In [ ]:
import math

# GRADED FUNCTION: initialize
def initialize(states, tag_counts, A, B, corpus, vocab):
    '''
    Input: 
        states: a list of all possible parts-of-speech
        tag_counts: a dictionary mapping each tag to its respective count
        A: Transition Matrix of dimension (num_tags, num_tags)
        B: Emission Matrix of dimension (num_tags, len(vocab))
        corpus: a sequence of words whose POS is to be identified in a list 
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output:
        best_probs: matrix of dimension (num_tags, len(corpus)) of floats
        best_paths: matrix of dimension (num_tags, len(corpus)) of integers
    '''
    
    sorted_tags = sorted(states)

    #### Your code goes here ####

    # Getting the total no. of POS tags
    n_tags = len(tag_counts)
    
    # Initializing the best_probs matrix 
    best_probs = np.zeros((n_tags, len(corpus)))
    
    # Initializing the best_paths matrix
    best_paths = np.zeros((n_tags, len(corpus)), dtype=int)
    
    # Initializing the start state, denoted by '--s--'
    s_idx = states.index("--s--")
    
    # Going through each of the POS tags
    for i in range(n_tags): 

        if A[s_idx,i]==0:
            
            # Initialize best_probs at POS tag 'i', column 0, to negative infinity
            best_probs[i,0] = float('-inf')
        
        else:
            # using the formula
            best_probs[i,0] = math.log(A[s_idx,i]) + math.log(B[i,vocab[corpus[0]]])

           
    return best_probs, best_paths

In [ ]:
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)

In [ ]:
# Test the function
print(f"best_probs[0,0]: {best_probs[0,0]:.4f}") 
print(f"best_paths[2,3]: {best_paths[2,3]:.4f}")

best_probs[0,0]: -22.6098
best_paths[2,3]: 0.0000


#### Expected Output: 

```CPP
best_probs[0,0]: -22.6098
best_paths[2,3]: 0.0000
```

### Task 1.4.2 Viterbi Forward Implementation (10 Points)


In this part of the assignment, you will implement the `viterbi_forward` segment. In other words, you will populate your `best_probs` and `best_paths` matrices.
- Walk forward through the corpus.
- For each word, compute a probability for each possible tag. 

Store the best_path and best_prob for every possible tag for each word in the matrices `best_probs` and `best_tags` using the pseudo code below.

`for each word in the corpus

    for each POS tag type that this word may be
    
        for POS tag type that the previous word could be
        
            compute the probability that the previous word had a given POS tag, that the current word has a given POS tag, and that the POS tag would emit this current word.
            
            retain the highest probability computed for the current word
            
            set best_probs to this highest probability
            
            set best_paths to the index 'k', representing the POS tag of the previous word which produced the highest probability `

In [ ]:
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab, corpus):
    '''
    Input: 
        A, B: The transiton and emission matrices respectively
        test_corpus: a list containing a preprocessed corpus
        best_probs: an initilized matrix of dimension (num_tags, len(corpus))
        best_paths: an initilized matrix of dimension (num_tags, len(corpus))
        vocab: a dictionary where keys are words in vocabulary and value is an index 
    Output: 
        best_probs: a completed matrix of dimension (num_tags, len(corpus))
        best_paths: a completed matrix of dimension (num_tags, len(corpus))
    '''
    # Retreiving the number of  POS tags
    num_tags = best_probs.shape[0]
    
    for i in range(1, len(test_corpus)): # word 0 was used to initialize
        
        for j in range(num_tags): 
            
            # Initializing the best_prob and best_path for word i 
            best_prob_i = float('-inf')
            best_path_i = None

            # For each POS tag that the previous word can be:
            for k in range(num_tags): 

                prob = best_probs[k,i-1] + math.log(A[k,j])+math.log(B[j,vocab[test_corpus[i]]]) #calculating the prob

                if prob > best_prob_i:
                    
                    # if the prob is greater than best prob then set it to best prob
                    best_prob_i = prob
                    #set best path to that path
                    best_path_i = k

            #storing the best prob and path after looking through all the tags
            best_probs[j,i] = best_prob_i
            best_paths[j,i] = best_path_i


    return best_probs, best_paths

In [1]:
# this will take a few minutes to run => processes ~ 30,000 words
best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab, prep)

In [ ]:
# Test this function 
print(f"best_probs[0,1]: {best_probs[0,1]:.4f}") 
print(f"best_probs[0,4]: {best_probs[0,4]:.4f}") 

best_probs[0,1]: -24.7822
best_probs[0,4]: -49.5601


#### Expected Output: 

```CPP
best_probs[0,1]: -35.2828
best_probs[0,4]: -54.4040
```

### Task 1.4.3 Viterbi Backward Implementation (10 Points)

<a name='1.4'></a>
## Part 1.4 Viterbi backward

Now you will implement the Viterbi backward algorithm.
- The Viterbi backward algorithm gets the predictions of the POS tags for each word in the corpus using the `best_paths` and the `best_probs` matrices.

The example below shows how to walk backwards through the best_paths matrix to get the POS tags of each word in the corpus. Recall that this example corpus has three words: "Loss tracks upward".

POS tag for 'upward' is `RB`
- Select the the most likely POS tag for the last word in the corpus, 'upward' in the `best_prob` table.
- Look for the row in the column for 'upward' that has the largest probability.
- Notice that in row 28 of `best_probs`, the estimated probability is -34.99, which is larger than the other values in the column.  So the most likely POS tag for 'upward' is `RB` an adverb, at row 28 of `best_prob`. 
- The variable `z` is an array that stores the unique integer ID of the predicted POS tags for each word in the corpus.  In array z, at position 2, store the value 28 to indicate that the word 'upward' (at index 2 in the corpus), most likely has the POS tag associated with unique ID 28 (which is `RB`).
- The variable `pred` contains the POS tags in string form.  So `pred` at index 2 stores the string `RB`.


POS tag for 'tracks' is `VBZ`
- The next step is to go backward one word in the corpus ('tracks').  Since the most likely POS tag for 'upward' is `RB`, which is uniquely identified by integer ID 28, go to the `best_paths` matrix in column 2, row 28.  The value stored in `best_paths`, column 2, row 28 indicates the unique ID of the POS tag of the previous word.  In this case, the value stored here is 40, which is the unique ID for POS tag `VBZ` (verb, 3rd person singular present).
- So the previous word at index 1 of the corpus ('tracks'), most likely has the POS tag with unique ID 40, which is `VBZ`.
- In array `z`, store the value 40 at position 1, and for array `pred`, store the string `VBZ` to indicate that the word 'tracks' most likely has POS tag `VBZ`.

POS tag for 'Loss' is `NN`
- In `best_paths` at column 1, the unique ID stored at row 40 is 20.  20 is the unique ID for POS tag `NN`.
- In array `z` at position 0, store 20.  In array `pred` at position 0, store `NN`.


In [ ]:
# GRADED FUNCTION: viterbi_backward
def viterbi_backward(best_probs, best_paths, corpus, states):
    '''
    This function returns the best path.
    
    '''
    maxProb = float('-inf')
    maxIndex = 0
    
    #### Your Code Goes Here ####

    # Getting the no. of words
    m = best_paths.shape[1] 
    # Initializing an array for storing the unique ID of POS tags
    arr = [None] * m
    # Initializing prediction array
    pred = [None] * m
    
    # storing no. of unique POS tags
    n_tags = best_probs.shape[0]
    
    for k in range(n_tags):

        if best_probs[k,-1]>ProbM: 
            #if the probability is higher than the previous best prob , substitute the Maximum prob
            
            ProbM = best_probs[k,-1]
    
            # Storing the unique ID of the POS tag
            arr[m - 1] = k
            
    #fill the pred array using the unique id 
    pred[m - 1] = states[arr[m - 1]]
    
    ## Step 2 ##
    # going backward through the best_paths and finding the best path
    for i in range(m-1, 0, -1): 
        
        #getting the unique ID
        pos_tag_for_word_i = arr[i]
        arr[i - 1] = best_paths[pos_tag_for_word_i,i]
        
        #fill the pred
        pred[i - 1] = states[arr[i - 1]]
        

    return pred

In [ ]:
# Run and test your function
pred = viterbi_backward(best_probs, best_paths, prep, states)
m=len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

The prediction for pred[-7:m-1] is: 
 ['see', 'them', 'here', 'with', 'us', '.'] 
 ['VB', 'PRP', 'RB', 'IN', 'PRP', '.'] 

The prediction for pred[0:8] is: 
 ['DT', 'NN', 'POS', 'NN', 'MD', 'VB', 'VBN'] 
 ['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken']


#### Expected Output: 

```CPP
The prediction for pred[-7:m-1] is: 
 ['see', 'them', 'here', 'with', 'us', '.'] 
 ['VB', 'PRP', 'RB', 'IN', 'PRP', '.'] 

The prediction for pred[0:8] is: 
 ['DT', 'NN', 'POS', 'NN', 'MD', 'VB', 'VBN'] 
 ['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken']
```

### Predicting on the dataset and print accuracy

In [ ]:
print('The word is:', prep[3])
print('Your prediction is:', pred[3])
print('Your corresponding label y is: ', y[3])
print(f"Accuracy of the Viterbi algorithm is {compute_accuracy(pred, y):.4f}")

The word is: temperature
Your prediction is: NN
Your corresponding label y is:  temperature	NN

Accuracy of the Viterbi algorithm is 0.9531


#### Expected Output: 

```CPP
The word is: points
Your prediction is: VBZ
Your corresponding label y is:  points	NNS

Accuracy of the Viterbi algorithm is 0.9531
```

<a name='2'></a>
# Part 2: LSTMs for POS tagging - using PyTorch (40 Points)
In this part, We will be building a bidirectional LSTM network to train and inference POS tagging on UDPOS dataset.<br>

PyTorch makes it easy by abstracting most of the details that go in building,training and inferencing a neural network. We recommend going through every PyTorch function that this notebook uses to gain more understanding.   

If you need a refresher or have never worked with Neural Networks before, here are a few resources:
- https://web.stanford.edu/~jurafsky/slp3/7.pdf
- https://web.stanford.edu/~jurafsky/slp3/9.pdf
- https://colah.github.io/posts/2015-08-Understanding-LSTMs/

We will be using PyTorch for defining, training and inferencing a neural network for our POS Tagging problem. If you have not used any deep learning framework/library, we recommend you spend some time understanding how to use these libraries. 

PyTorch Resources:
- https://pytorch.org/tutorials/
- https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html 

You will need the following imports. Install these libraries using the following commands. 
- Installing pytorch - https://pytorch.org/get-started/locally/ (choose your setup from here)
- conda install -c conda-forge spacy
- conda install -c pytorch torchtext

Training a neural network model will take time. 
- Make use of your **Nvidia** GPU if you have one. 
- If not, you can use Google Colab / Kaggle notebooks. You get a free GPU for a limited time to tweak your hyperparameters.
- Without a GPU, You might have to wait longer to experiment.

In [1]:
!pip install -U torch==1.8.0 torchtext==0.9.0

# Reload environment
exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 82.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.


In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim

# a package that provides processing utilities and popular datasets for natural language
from torchtext.legacy import data
from torchtext.legacy.datasets import UDPOS

import spacy
from tqdm import tqdm 
import random

In [2]:
# Using a seed to maintain consistent and reproducible results
import numpy as np
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
# This cell downloads and prepares data, a TorchText Dataset Object

TEXT = data.Field(lower = True)
UD_TAGS = data.Field(unk_token = None)
fields = (("text", TEXT), ("udtags", UD_TAGS))
train_data, valid_data, test_data = UDPOS.splits(fields)

downloading en-ud-v2.zip


en-ud-v2.zip: 100%|██████████| 688k/688k [00:01<00:00, 628kB/s]


extracting


### Visualizing the torchtext dataset

In [4]:
print("Length of the dataset", len(train_data))
for i in range(0,5):
    print("TEXT ", i+1 ,*(train_data[i].__dict__['text']))
    print("TAGS ", i+1 ,*(train_data[i].__dict__['udtags']))

Length of the dataset 12543
TEXT  1 al - zaman : american forces killed shaikh abdullah al - ani , the preacher at the mosque in the town of qaim , near the syrian border .
TAGS  1 PROPN PUNCT PROPN PUNCT ADJ NOUN VERB PROPN PROPN PROPN PUNCT PROPN PUNCT DET NOUN ADP DET NOUN ADP DET NOUN ADP PROPN PUNCT ADP DET ADJ NOUN PUNCT
TEXT  2 [ this killing of a respected cleric will be causing us trouble for years to come . ]
TAGS  2 PUNCT DET NOUN ADP DET ADJ NOUN AUX AUX VERB PRON NOUN ADP NOUN PART VERB PUNCT PUNCT
TEXT  3 dpa : iraqi authorities announced that they had busted up 3 terrorist cells operating in baghdad .
TAGS  3 PROPN PUNCT ADJ NOUN VERB SCONJ PRON AUX VERB ADP NUM ADJ NOUN VERB ADP PROPN PUNCT
TEXT  4 two of them were being run by 2 officials of the ministry of the interior !
TAGS  4 NUM ADP PRON AUX AUX VERB ADP NUM NOUN ADP DET PROPN ADP DET PROPN PUNCT
TEXT  5 the moi in iraq is equivalent to the us fbi , so this would be like having j. edgar hoover unwittingly employ a

### GloVe Vector initialization 
Vectorizing the input words is an impotant step in the NLP pipeline that can determine the end performance of neural networks. GloVe vectors capture both global statistics and local statistics of a corpus. We use GloVe to convert words to embeddings in the vector space based on their semantics. 

To learn more about GloVe please read the following resource:
- https://nlp.stanford.edu/pubs/glove.pdf

In [5]:
# the words should have atleast a min frequency of 2 to build its vocab
MIN_FREQ = 2

# Torch text builds the vocabulary based on word representations from glove. 
TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)


UD_TAGS.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:41, 5.33MB/s]                           
100%|█████████▉| 399999/400000 [00:14<00:00, 27192.36it/s]


In [9]:
# number of tags in the dataset
len(UD_TAGS.vocab)

18

##### Expected output
18

<a name='2.1'></a>
# Part 2.1: Building the neural network

We will make use of the GloVe embeddings and build a bi-directional LSTM. You will be able to tune the hyper parameters of the network and see what works. 

It involves duplicating the first recurrent layer in the network so that there are now two layers side-by-side, then providing the input sequence as-is as input to the first layer and providing a reversed copy of the input sequence to the second.

The idea is to split the state neurons of a regular RNN in a part that is responsible for the positive time direction (forward states) and a part for the negative time direction (backward states)

More on it here: https://maxwell.ict.griffith.edu.au/spl/publications/papers/ieeesp97_schuster.pdf

All the internal computations/details will be taken care by PyTorch. You will be able to implement many variations of this neural networks with minor changes in code. Expect your neural network definition to be under 10 lines.

Your PyTorch model (inherits torch.nn.Module) definition contains defining two functions:
    -Init : Which specifies what layers to initialize.
    -Forward: Which defines the order of computations in these layers. <br>
**Note** - We will not grade based on accuracy, We grade if your model converges. You can follow your order of code, if you think the comments are not helping.  

### Part 2.1.2 Building LSTM network - 20 Points

In [60]:
class LSTMPOSTagger(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim, 
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
        
        # Define an embedding layer that converts the words to embeddings based on GloVe.
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)

        
        # Define a bi-directional LSTM layer with the hyperparameters. 
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        
        # Define a dropout layer that helps in regularization
        

        
        # Define a Linear layer which can associate lstm output to the final output 
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)
        
        
        
    def forward(self, text):
        # pass text through embedding layer
        embedded = self.dropout(self.embedding(text))
        
        # pass embeddings into LSTM
        lstm_out, (hidden, cell) = self.lstm(embedded)
        
        # pass the LSTM output to dropout and fully connected linear layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        # we use our outputs to make a prediction of what the tag should be
        predictions = out
        
        # predictions = [sent len, batch size, output dim]
        
        return predictions

In [61]:
# Tweak the Nones
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = len(UD_TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTMPOSTagger(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS, 
                        BIDIRECTIONAL, 
                        DROPOUT, 
                        PAD_IDX)

In [62]:
# initializing model weights for better convergence
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, std=0.1)
model.apply(init_weights)

# initializing model embeddings with glove word vectors
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

# making the padding embeddings as all zero, as we don't want to learn paddings.
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [63]:
# If your PC doesn't have enough CPU Ram or Video memory, try decreasing the batch_size
BATCH_SIZE = 128
device = torch.device('cpu')

In [64]:
# BucketIterator allows for data to be split into buckets of equal size,
# any remaining space is filled with pad token
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]

### Optimizer and Loss Function
Optimizers are algorithms or methods used to change the attributes of the neural network such as weights and learning rate to reduce the losses. Optimizers are used to solve optimization problems by minimizing the function.
- PyTorch provides many Optimizer Algorithms, feel free to try them and the one that works best for you. 
- Link - https://pytorch.org/docs/stable/optim.html
- We will be using CrossEntropyLoss as predicting a word tag is a classification problem.

In [65]:
# optimizer to train the model
optimizer = optim.Adam(model.parameters(), lr=0.001)
# ignoring the padding in our loss calculation
criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [66]:
device

device(type='cpu')

In [67]:
# use gpu if available, These lines move your model to gpu from cpu if available
model = model.to(device)
criterion = criterion.to(device)

# If this line prints cuda, your machine is equipped with a Nvidia GPU and PyTorch is utilizing the GPU
print(device)

cpu


In [68]:
# method to check for accurcy of the model ignoring the pad index
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != TAG_PAD_IDX).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / y[non_pad_elements].shape[0]

### Part 2.1.2 - Training and Testing LSTM - 20 Points
- Decide your epochs to train based on loss and accuracy
- Fill single line PyTorch commands. 

In [78]:
EPOCHS = 10

for epoch in tqdm(range(EPOCHS)):
    model.train()
    train_epoch_loss = 0
    train_epoch_acc = 0
    print(f'Epoch: {epoch+1:02}\n')
    for batch in train_iterator:
        
        # returns a batch of text to train on (sent len, batch size)
        text = batch.text
        tags = batch.udtags
        
        # Add a command that makes the optimizer with zero gradients for each iteration 
         # Add code line
        optimizer.zero_grad()

        
        # Add a command that feeds the batch to the model
         # Add code line
        predictions = model(text)
        
        # predictions = (sent len, batch size, output dim)
          # tags = (sent len, batch size)
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        # Add a command that calculates loss
         # Add code line
        train_loss = criterion(predictions, tags)
                
        
        
        train_loss.backward()
        
        optimizer.step() 

        
        # Make use of the categorical accuracy and calculate accuracy
         # Add code line
        train_acc = categorical_accuracy(predictions, tags)
        
        # Add a command that calculates gradients
         # Add code line
        

        
        # Add a command that updates the weights by taking steps 
         # Add code line

        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()   
            
        
        # Calculate loss
    print(f'\t [Train Loss] : {train_loss:.3f} | [Train Acc] : {train_acc*100:.2f}%\n')
    
    val_epoch_loss = 0
    val_epoch_acc = 0
    
    
    
    # Add a command that moves the model to validation mode
    model.eval()
    
    with torch.no_grad():
    
        for batch in valid_iterator:

            text = batch.text
            tags = batch.udtags
        
            # Add the same command that feeds the batch to the model
            # Add code line
            predictions = model(text)

            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            # Add the same command that calculates loss
            # Add code line
            val_loss = criterion(predictions, tags)

            # Make use of the categorical accuracy function and calculate accuracy
             # Add code line
            val_acc = categorical_accuracy(predictions, tags) 
            
            # Calculate validation loss
        print(f'\t [Val Loss] : {val_loss:.3f} | [Val Acc] : {val_acc*100:.2f}%\n')

      

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 01

	 [Train Loss] : 0.311 | [Train Acc] : 90.07%



 10%|█         | 1/10 [01:23<12:35, 83.89s/it]

	 [Val Loss] : 0.318 | [Val Acc] : 89.51%

Epoch: 02

	 [Train Loss] : 0.248 | [Train Acc] : 92.13%



 20%|██        | 2/10 [02:45<11:00, 82.58s/it]

...

	 [Val Loss] : 0.259 | [Val Acc] : 91.58%

Epoch: 09

	 [Train Loss] : 0.150 | [Train Acc] : 94.88%



 90%|█████████ | 9/10 [12:16<01:21, 81.45s/it]

	 [Val Loss] : 0.250 | [Val Acc] : 91.75%

Epoch: 10

	 [Train Loss] : 0.147 | [Train Acc] : 95.08%



100%|██████████| 10/10 [13:38<00:00, 81.86s/it]

	 [Val Loss] : 0.244 | [Val Acc] : 92.26%



In [79]:
# testing the accuracy on test set
test_acc=0
model.eval()

# Computes without the gradients. Use this while testing your model.
# As we do not intend to learn from the data
with torch.no_grad():
    for batch in test_iterator:


      text = batch.text
      tags = batch.udtags

    # Add the same command that feeds the batch to the model
    # Add code line
      predictions = model(text)
      
      predictions = predictions.view(-1, predictions.shape[-1])
      tags = tags.view(-1)

    # Make use of the categorical accuracy function and calculate accuracy
    # Add code line
      final_acc = categorical_accuracy(predictions, tags) 
    
    # Calculate Accuracy
print(f'Test Acc: {final_acc*100:.2f}%\n')

Test Acc: 91.65%



***Note***: You are using a different dataset compared to part 1 and 2. This part of the assignment is designed/aimed to help develop basic understanding of Neural Networks. Although we expect an accuracy of above 85%, we do not grade based on the accuracy output. 

In [80]:
# Try different inputs to these function.
def test_lstm(test_sentence):
    x= test_sentence.unsqueeze(-1).to(device)
    pred = model(x)
    pred = pred.argmax(-1)
    pred_tags = [UD_TAGS.vocab.itos[t.item()] for t in pred]
    true_tags = [UD_TAGS.vocab.itos[t.item()] for t in test_labels]
    tokenized_sentence = [TEXT.vocab.itos[t.item()] for t in test_sentence]
    return tokenized_sentence, true_tags,pred_tags    

In [81]:
test_sentence = text[0]
test_labels = tags[0:len(test_sentence)]
print(test_lstm(test_sentence)[0])
print("True tags", test_lstm(test_sentence)[1])
print("Predicted Tags",test_lstm(test_sentence)[2])

['(', 'stay', 'important', 'the', '<unk>', 'i', 'did', '<unk>', 'the', 'here', 'we', 'as', 'i', 'on', '<unk>', 'it', 'it', 'just', '"', 'the', 'the', '"', 'if', 'the', 'any', 'in', 'in', 'over', 'it']
True tags ['PUNCT', 'VERB', 'ADJ', 'DET', 'ADV', 'PRON', 'AUX', 'PROPN', 'DET', 'ADV', 'PRON', 'SCONJ', 'PRON', 'ADP', 'PROPN', 'PRON', 'PRON', 'ADV', 'PUNCT', 'DET', 'DET', 'PUNCT', 'SCONJ', 'DET', 'DET', 'ADP', 'ADP', 'ADV', 'PRON']
Predicted Tags ['PUNCT', 'VERB', 'ADJ', 'DET', 'NOUN', 'PRON', 'AUX', 'VERB', 'DET', 'ADV', 'PRON', 'SCONJ', 'PRON', 'ADP', 'VERB', 'PRON', 'PRON', 'ADV', 'PUNCT', 'DET', 'DET', 'PUNCT', 'SCONJ', 'DET', 'DET', 'ADP', 'ADV', 'ADP', 'PRON']


In [82]:
# Save your model if it performs well. This saves all the trained weights,
# so that you don't have to train again in your codewalk
torch.save(model.state_dict(), "./LSTMPOSTAG.pth")

# loading?l
# model.load_state_dict(torch.load("./LSTMPOSTAG.pth"))

### Theory Questions: 10 Points
*** Q1. Give some real word examples where POS tagging is used   ***<br>
*** Q2. What are the hidden variables in HMM in this assignment? Why are they called hidden? *** <br>
*** Q3. How Viterbi Algorithm provides more efficient estimation compared to brute force calculation of all tag combinations? *** <br>

# **Q1**

POS tagging can be used for:

1) Identifying **information** in text like name,location,organization.

2) Used for creating the customer profiles or for identification of important people in an article .

3) Text categorization - spam.

4) Translate text by recognizing the grammatical structure of languages.

## **Q2)**
The state, i.e ,which state we go to next/ sequence of states is hidden,In our assignment it is the **POS tag** of that word.

They are called hidden because at any point of time, the sequence of states through which the model passes is hidden and could not be observed . We can deduce just the probability of a sequence of states.

# **Q3)**
The viterbi algorithm prunes candidates that cannot have the highest probability, without fully forming the path,thus saving time.
 It finds the highest probability for a word against all tags by going through the transmission and emission probabilities, multiplying these probabilities, and then finding the maximum probability. Thus coming to a solution more efficiently than the brute force.